In [1]:
#import all required libraries

import os
import cv2
import glob
import numpy as np
from skimage.feature import hog
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report 

In [2]:
#path to the dataset

path='Dataset/*'

In [3]:
#read multiple file

data_path = os.path.join(path,'*g')
imagePaths = glob.glob(data_path)
print(len(imagePaths))

250


In [5]:
#  define a function for image segmentation
# for extraction of green and brown color

def img_segmentation(image):
    rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    hsv_img = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2HSV)
    lower_green = np.array([25,0,20])
    upper_green = np.array([100,255,255])
    healthy_mask = cv2.inRange(hsv_img, lower_green, upper_green)
    result = cv2.bitwise_and(rgb_img,rgb_img, mask=healthy_mask)
    lower_brown = np.array([10,0,10])
    upper_brown = np.array([30,255,255])
    disease_mask = cv2.inRange(hsv_img, lower_brown, upper_brown)
    disease_result = cv2.bitwise_and(rgb_img, rgb_img, mask=disease_mask)
    final_mask = healthy_mask + disease_mask
    final_result = cv2.bitwise_and(rgb_img, rgb_img, mask=final_mask)
    return final_result

In [6]:
#declare two empty lists for labels and features

labels = []
hogFeatures = []

In [7]:
# loop over the input images 

print('Wait, I am performing preprocessing steps and extracting HOG features...')

for imagePath in imagePaths:
    
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    
    # resize image to 256*256
    image = cv2.resize(image, (256,256)) 
    
    # perform image sagmentation
    segmented_img   = img_segmentation(image)
    
    # extract HOG features
    features, hog_image = hog(segmented_img, orientations=8, pixels_per_cell=(8, 8),
                    cells_per_block=(1, 1), visualize=True, multichannel=True)
    
    # store features
    hogFeatures.append(features)
    
    #extract labels
    label = imagePath.split(os.path.sep)[-2].split('_')
    labels.append(label)
    
print('I am Done.')

Wait, I am performing preprocessing steps and extracting HOG features...
I am Done.


In [8]:
#split the data to train and test formate

X_train, X_test, y_train, y_test = train_test_split(hogFeatures, labels, test_size = 0.3 )

In [9]:
# Defining all classifires

#Create a svm Classifier
model_SVM = svm.SVC(kernel='linear') # Linear Ke

#Create a Gaussian Classifier
model_NB = GaussianNB()

# Create Decision Tree classifer object
model_DT = DecisionTreeClassifier()

# Create Knn classifer object
model_KNN = KNeighborsClassifier(n_neighbors=3)

#Create a RF Classifier
model_RF = RandomForestClassifier(n_estimators=100)

#Create a LR Classifier
model_LR = LogisticRegression(solver="lbfgs", multi_class="auto")

In [10]:
#Train the SVM model using the training sets

model_SVM.fit(X_train, np.ravel(y_train))

SVC(kernel='linear')

In [11]:
#Train the NB model using the training sets

model_NB.fit(X_train, np.ravel(y_train))

GaussianNB()

In [12]:
#Train the DT model using the training sets

model_DT.fit(X_train, np.ravel(y_train))

DecisionTreeClassifier()

In [13]:
#Train the knn model using the training sets

model_KNN.fit(X_train, np.ravel(y_train))

KNeighborsClassifier(n_neighbors=3)

In [14]:
#Train the RF model using the training sets

model_RF.fit(X_train, np.ravel(y_train))

RandomForestClassifier()

In [15]:
#Train the LR using the training sets

model_LR.fit(X_train, np.ravel(y_train))

C:\Users\Umar Farooq\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [16]:
# prediction on test data 

#Predict the response for test dataset 
SVM_pred = model_SVM.predict(X_test)

#Predict the response for test dataset 
NB_pred = model_NB.predict(X_test)

#Predict the response for test dataset 
DT_pred = model_DT.predict(X_test)

#Predict the response for test dataset 
KNN_pred = model_KNN.predict(X_test)

#Predict the response for test dataset 
RF_pred = model_RF.predict(X_test)

#Predict the response for test dataset 
LR_pred = model_LR.predict(X_test)

In [17]:
# Model Accuracy: how often is the classifier correct?
print(" SVM Accuracy:", metrics.accuracy_score (y_test, SVM_pred))

# Model Accuracy: how often is the classifier correct?
print("NB Accuracy:", metrics.accuracy_score (y_test, NB_pred))

# Model Accuracy: how often is the classifier correct?
print("DT Accuracy:", metrics.accuracy_score (y_test, DT_pred))

# Model Accuracy: how often is the classifier correct?
print("KNN Accuracy:", metrics.accuracy_score (y_test, KNN_pred))

# Model Accuracy: how often is the classifier correct?
print("RF Accuracy:", metrics.accuracy_score (y_test, RF_pred))

# Model Accuracy: how often is the classifier correct?
print("LR Accuracy:", metrics.accuracy_score (y_test, LR_pred))

 SVM Accuracy: 0.84
 NB Accuracy: 0.4666666666666667
DT Accuracy: 0.49333333333333335
KNN Accuracy: 0.72
RF Accuracy: 0.7333333333333333
LR Accuracy: 0.7733333333333333


In [18]:
# Classification Report

target_names=["Bacterial spot","Healthy","Spote spider mites","Mosaic virus","Yellow leaf curlr virus"]

# Classification Report for SVM
print('Classification Report of Support Vector Machine ')
print('\n')
print(classification_report(y_test, SVM_pred, target_names=target_names))
print('****************************************************************')

# Classification Report for Gaussian Naïve Bayes Classifier
print('Classification Report of Naïve Bayes ')
print('\n')
print(classification_report(y_test, NB_pred, target_names=target_names))
print('****************************************************************')

# Classification Report for SVM
print('Classification Report of Decision Tree ')
print('\n')
print(classification_report(y_test, DT_pred, target_names=target_names))
print('****************************************************************')

# Classification Report for K-Nearest Neighbours
print('Classification Report of K-Nearest Neighbours ')
print('\n')
print(classification_report(y_test, KNN_pred, target_names=target_names))
print('****************************************************************')

# Classification Report for Random Forest
print('Classification Report of Random Forest ')
print('\n')
print(classification_report(y_test, RF_pred, target_names=target_names))
print('****************************************************************')

# Classification Report for Random Forest
print('Classification Report of Logistic Regression ')
print('\n')
print(classification_report(y_test, LR_pred, target_names=target_names))
print('****************************************************************')

Classification Report of Support Vector Machine 


                         precision    recall  f1-score   support

         Bacterial spot       0.89      1.00      0.94        16
                Healthy       0.67      1.00      0.80        16
     Spote spider mites       0.87      0.93      0.90        14
           Mosaic virus       1.00      0.35      0.52        17
Yellow leaf curlr virus       1.00      1.00      1.00        12

               accuracy                           0.84        75
              macro avg       0.88      0.86      0.83        75
           weighted avg       0.88      0.84      0.82        75

****************************************************************
Classification Report of Naïve Bayes 


                         precision    recall  f1-score   support

         Bacterial spot       0.36      0.75      0.49        16
                Healthy       0.31      0.25      0.28        16
     Spote spider mites       0.83      0.36      0.50      

In [19]:
# performing prediction on images
# loop over the input images

for img in glob.glob('Validation/*'):
    
    image= cv2.imread(img)
    image = cv2.resize(image, (256,256)) 
    segmented_img   = img_segmentation(image)  

    features, hog_image = hog(segmented_img, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(1, 1), visualize=True, multichannel=True)

    #cv2.imshow('',img1)
    features=features.reshape(1, -1)
      
    # font shape and style
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (1, 20) 
    fontScale = 0.5
    color = (0, 0, 255) 
    thickness = 1
    
    # Testing images using SVM trained model
    result = model_SVM.predict(features)

    # Using cv2.putText() method 
    result=str(result)
    print(result)
    
    image = cv2.putText(image,result, org, font, fontScale, 
            color, thickness, cv2.LINE_AA, False)  
    
  
    # Displaying the image 
    cv2.imshow('', image)  
    cv2.waitKey(0)
    cv2.destroyAllWindows()

['Tomato yellow leaf curl virus']
['Tomato mosaic virus']
['Tomato yellow leaf curl virus']
['Tomato yellow leaf curl virus']
['Tomato mosaic virus']
['Tomato bacterial spot']
['Tomato yellow leaf curl virus']
['Tomato mosaic virus']
['Tomato spotted spider mite']
['Tomato healthy']
['Tomato spotted spider mite']
['Tomato bacterial spot']
['Tomato bacterial spot']
['Tomato mosaic virus']
['Tomato bacterial spot']
['Tomato healthy']
['Tomato healthy']
['Tomato healthy']
['Tomato healthy']
['Tomato mosaic virus']
